# MT Bias Estimation of the variable Flip Angle $T_1$-Mapping Method
This notebooks estimates the bias of $T_1$ caused by the magnetization transfer effect in the variable flip angle method. 

First, we load all required packages:

In [12]:
using MRIgeneralizedBloch
using StaticArrays

Define the point in parameter space at which the bias will be calculated:

In [13]:
m0s = 0.1
R1a = 0.625 # 1/s
R1f = R1a   # 1/s
R1s = R1a   # 1/s
R2f = 15    # 1/s
Rx  = 30    # 1/s
T2s = 10e-6 # s
R2slT = precompute_R2sl()
ω0 = 0      # rad/s
B1 = 1;     # in units of B1_nominal

In the following cell, we define the sequence parameters as suggested in [Yarnykh 2016](https://doi.org/10.1002/mrm.25811) and start with the first flip angle of 4deg.

In [14]:
α1 = 4 * π/180
TRF = 1e-3 # s
TR = 21e-3;    # s

To simulate the steady state magnetization, we calculate the propagator matrices `u_sp` for the (perfect) spoiler, `u_exc` for the excitation pulse, and `u_fp` for free precession:

In [15]:
R2sl = R2slT[1](TRF, α1, B1, T2s)
u_sp =
    @SMatrix [
        0 0 0 0 0 0;
        0 0 0 0 0 0;
        0 0 1 0 0 0;
        0 0 0 0 0 0;
        0 0 0 0 1 0;
        0 0 0 0 0 1]

u_exc = exp(hamiltonian_linear(α1/TRF, B1, ω0, TRF, m0s, R1f, R2f, Rx, R1s, R2sl))
u_fp  = exp(hamiltonian_linear(0, B1, ω0, TR - TRF, m0s, R1f, R2f, Rx, R1s, R2sl));

The steady state magnetization is is given by solving the linear equation $m_{ss} = U m_{ss}$ or, more precisely, by solving $U m_{ss} - A_0 m_{ss} = C$, where $A_0$ is a unity matrix modified to have `0` as the last diagonal matrix and $C$ is a zero vector modified to have `1` as the last entry. 

In [16]:
U =  u_exc * u_sp * u_fp
Q = U - MRIgeneralizedBloch.A0(nothing)
m1 = Q \ MRIgeneralizedBloch.C(nothing);

We repeat the same simulation for a flip angle of 25deg:

In [17]:
α2 = 25 * π/180
R2sl = R2slT[1](TRF, α2, B1, T2s)

u_exc = exp(hamiltonian_linear(α2/TRF, B1, ω0,  TRF, m0s, R1f, R2f, Rx, R1s, R2sl))
u_fp  = exp(hamiltonian_linear(0, B1, ω0,  TR - TRF, m0s, R1f, R2f, Rx, R1s, R2sl))
U =  u_exc * u_sp * u_fp
Q = U - MRIgeneralizedBloch.A0(nothing)
m2 = Q \ MRIgeneralizedBloch.C(nothing);

And estimate the apparent longitudinal relaxation rate $R_1^a$:

In [18]:
ρ = m2[1] / m1[1]
fρ = (ρ * sin(α1) * cos(α2) - cos(α1) * sin(α2)) / (ρ * sin(α1) - sin(α2))
R1a_est = log(fρ) / TR

0.6864117207059647

Which we can compare to the actual apparent relaxation rate $R_1^a$:

In [19]:
R1a_est / R1a - 1

0.09825875312954357